## Sentiment analysis

In [ ]:
import mysql.connector
from transformers import pipeline
import numpy as np
from credentials import ipCred, usernameCred, passwordCred, databaseCred
# Initialize the sentiment pipeline with FinBERT.
# Make sure you have installed the required libraries (transformers, torch)
classifier = pipeline("sentiment-analysis", model="ProsusAI/finbert")

db_config = {
    'host': ipCred,
    'user': usernameCred,
    'password': passwordCred,
    'database': databaseCred
}

conn = mysql.connector.connect(**db_config)

# --- Cursor A: Fetch rows that need sentiment scores
cursor_fetch = conn.cursor()
fetch_query = """
    SELECT news_id, summary
    FROM TSLA_news
    WHERE sentiment IS NULL
    LIMIT 5000;
"""
cursor_fetch.execute(fetch_query)
rows = cursor_fetch.fetchall()
cursor_fetch.close()  # Close fetch cursor

# --- Cursor B: Update rows with sentiment scores
cursor_update = conn.cursor()
update_query = """
    UPDATE TSLA_news
    SET sentiment = %s
    WHERE news_id = %s
"""

for (news_id, summary) in rows:
    if not summary:
        sentiment_score = None
    else:
        # Use FinBERT to classify the summary.
        # FinBERT typically outputs three labels: NEGATIVE, NEUTRAL, POSITIVE.
        result = classifier(summary[:512])
        # Initialize scores
        p_pos = 0.0
        p_neg = 0.0
        # Iterate over results to pick out the scores for positive and negative.
        for r in result:
            label = r['label'].upper()
            score = r['score']
            if label == 'POSITIVE':
                p_pos = score
            elif label == 'NEGATIVE':
                p_neg = score
        # Compute bipolar sentiment: positive minus negative.
        sentiment_score = p_pos - p_neg

    cursor_update.execute(update_query, (sentiment_score, news_id))

conn.commit()
cursor_update.close()
conn.close()

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
